# 1. Lecture CSV

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_val_score, GridSearchCV

In [2]:
df = pd.read_csv("../datas/Walmart_Store_sales_clean.csv")
df.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day
0,6,1572117.54,0,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0
1,13,1807545.43,0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0
2,11,1244390.03,0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN
3,6,1644470.66,0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0
4,4,1857533.70,0,NaN,2.756,126.160226,7.896,2010.0,5.0,28.0


---

# 2. Fight overfitting

## 2.1 Découpage stratifié des données

Nous avons vu, d'après les coefficients de regression, que la variable qui a le plus d'influence sur la valeur cible est `Store`. 

Je vais donc, dans un premier temps, m'assurer que la proportion des différents magasins reste la même dans la répartition des jeux de données.

In [3]:
target = "Weekly_Sales"

x = df.drop(target, axis=1)
y = df[target]

print(x.head)
print(y)

<bound method NDFrame.head of      Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
0        6             0        59.61       3.045  214.777523         6.858   
1       13             0        42.38       3.435  128.616064         7.470   
2       11             0        84.57         NaN  214.556497         7.346   
3        6             0        78.89       2.759  212.412888         7.092   
4        4             0          NaN       2.756  126.160226         7.896   
..     ...           ...          ...         ...         ...           ...   
126     14             0        72.62       2.780  182.442420         8.899   
127      7             0        20.74       2.778         NaN           NaN   
128     17             0        57.14       2.841  126.111903           NaN   
129      8             0        86.05       3.638  219.007525           NaN   
130     19             0        55.20       4.170  137.923067         8.150   

       Year  Month   

Je stratifie sur le Magasin

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=x["Store"])

In [5]:
numerical_columns = ["Temperature", "Fuel_Price", "CPI", "Unemployment", "Year", "Month", "Day"]
categorical_columns = ["Store", "Holiday_Flag"]

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(
    steps=[
    ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore"))
    ])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numerical_columns),
        ("cat", categorical_transformer, categorical_columns)
    ])

x_train = preprocessor.fit_transform(x_train)
x_test = preprocessor.transform(x_test) 

In [6]:
lr = LinearRegression()
lr.fit(x_train, y_train)

# Dans un modèle de régression linéaire, on utilise le score R2. Ce score analyse dans quelle mesure les prédictions sont proches des valeurs réelles.
print("R2 score training :", lr.score(x_train, y_train))
print("R2 score test :",  lr.score(x_test, y_test))

R2 score training : 0.9720924485232406
R2 score test : 0.9514792505670808


La stratification a aidé à obtenir un score de test beaucoup plus proche du score d'entraînement. On observe moins d’overfitting.

In [7]:
scores = cross_val_score(lr, x_train, y_train, cv = 10)

print("The cross-validated R2-score is : ", scores.mean())
print("The standard deviation is : ", scores.std())

The cross-validated R2-score is :  0.936963848548281
The standard deviation is :  0.034551806447655375


Le score moyen (0.93) se rapproche du score sur le test stratifié (0.95), ce qui confirme la bonne généralisation du modèle.

L’écart-type est faible (0.03), ça veut dire que la performance du modèle est stable sur les différentes partitions de jeux d'entraînement pendant la validation croisée.

---

Sauvegarde des résultats dans un Dataframe

In [8]:
scores_df = pd.DataFrame(columns = ["model", "R2"])
new_rows = [{"model": "baseline_stratify", "R2": lr.score(x_test, y_test)}]
scores_df = pd.concat([scores_df, pd.DataFrame(new_rows)], ignore_index=True)
scores_df.to_csv("../datas/Walmart_Scores.csv", mode="a", header=False, index=False)

/tmp/ipykernel_1901/2616532116.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  scores_df = pd.concat([scores_df, pd.DataFrame(new_rows)], ignore_index=True)


---

## 2.2 Ridge et Lasso

Ridge et Lasso sont des techniques de régularisation qui ajoutent une pénalité aux coefficients d'un modèle de régression linéaire, afin d'éviter le surapprentissage (overfitting).

#### Ridge

In [9]:
ridge = Ridge()
ridge.fit(x_train, y_train)

# Dans un modèle de régression linéaire, on utilise le score R2. Ce score analyse dans quelle mesure les prédictions sont proches des valeurs réelles.
print("R2 score training :", ridge.score(x_train, y_train))
print("R2 score test :",  ridge.score(x_test, y_test))

R2 score training : 0.9334419096463248
R2 score test : 0.8907935192529417


In [10]:
scores = cross_val_score(ridge, x_train, y_train, cv = 10)

print("The cross-validated R2-score is : ", scores.mean())
print("The standard deviation is : ", scores.std())

The cross-validated R2-score is :  0.8606127835985309
The standard deviation is :  0.04322351203557292


Avec Ridge :
- Le score sur le set d’entraînement diminue par rapport au modèle de base, ce qui indique une réduction de l’overfitting. Cependant, un surapprentissage persiste, avec un écart d’environ 0.04 entre les scores d’entraînement et de test. L'overfitting est donc réduit mais moins qu'avec le modèle de régression linéaire stratifié.
- Le score moyen obtenu par validation croisée se rapproche du score de test, ce qui renforce la fiabilité du modèle. Toutefois, ce score reste relativement bas.

#### Lasso

In [11]:
lasso = Lasso()
lasso.fit(x_train, y_train)

# Dans un modèle de régression linéaire, on utilise le score R2. Ce score analyse dans quelle mesure les prédictions sont proches des valeurs réelles.
print("R2 score training :", lasso.score(x_train, y_train))
print("R2 score test :",  lasso.score(x_test, y_test))

R2 score training : 0.9720924457440111
R2 score test : 0.9514777614582287


In [12]:
scores = cross_val_score(lasso, x_train, y_train, cv = 10)

print("The cross-validated R2-score is : ", scores.mean())
print("The standard deviation is : ", scores.std())

The cross-validated R2-score is :  0.9369724491943223
The standard deviation is :  0.03454459556999177


Avec Lasso :
- Les scores d'entraînement et de test sont proches, ce qui indique une diminution de l’overfitting.
- Le score moyen en validation croisée est également proche du score de test, avec une faible variance (0.03), ce qui reflète une bonne stabilité du modèle.

Ces résultats étant très proches de ceux obtenus avec la régression linéaire stratifiée, l’intérêt d’utiliser une technique de régularisation comme Lasso reste limité dans ce cas.

---

Sauvegarde des résultats dans un dataframe

In [13]:
scores_df = pd.DataFrame(columns = ["model", "R2"])
new_rows = [{"model": "ridge", "R2": ridge.score(x_test, y_test)},
            {"model": "lasso", "R2": lasso.score(x_test, y_test)}]
scores_df = pd.concat([scores_df, pd.DataFrame(new_rows)], ignore_index=True)
scores_df.to_csv("../datas/Walmart_Scores.csv", mode="a", header=False, index=False)

/tmp/ipykernel_1901/1424266185.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  scores_df = pd.concat([scores_df, pd.DataFrame(new_rows)], ignore_index=True)


---

## 2.3 Bonus : Hyperparamètres

Pour chacune de ces techniques de régularisation, il faut trouver le paramètre optimum

In [14]:
# Je teste plusieurs alpha pour Ridge
regressor = Ridge()
params = {"alpha": [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]}

best_ridge = GridSearchCV(regressor, param_grid=params, cv=5)
best_ridge.fit(x_train, y_train)
print("Best hyperparameters : ", best_ridge.best_params_)
print("Best R2 score : ", best_ridge.best_score_)

Best hyperparameters :  {'alpha': 0.01}
Best R2 score :  0.9489277603372974


In [15]:
# Idem pour Lasso
regressor = Lasso(max_iter=2000)
params = {"alpha": [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]}

best_lasso = GridSearchCV(regressor, param_grid=params, cv=5)
best_lasso.fit(x_train, y_train)
print("Best hyperparameters : ", best_lasso.best_params_)
print("Best R2 score : ", best_lasso.best_score_)

Best hyperparameters :  {'alpha': 0.5}
Best R2 score :  0.9485252395495568


In [16]:
# Print R2 scores
print("RIDGE // R2 score on training set : ", best_ridge.score(x_train, y_train))
print("RIDGE // R2 score on test set : ", best_ridge.score(x_test, y_test))

print("---")

print("LASSO // R2 score on training set : ", best_lasso.score(x_train, y_train))
print("LASSO // R2 score on test set : ", best_lasso.score(x_test, y_test))

RIDGE // R2 score on training set :  0.972070656699971
RIDGE // R2 score on test set :  0.9514662487695481
---
LASSO // R2 score on training set :  0.9720924478283968
LASSO // R2 score on test set :  0.9514785068848646


In [17]:
scores_ridge = cross_val_score(best_ridge.best_estimator_, x_train, y_train, cv = 10)
print("RIDGE // The cross-validated R2-score is : ", scores_ridge.mean())
print("RIDGE // The standard deviation is : ", scores_ridge.std())

print("---")

scores_lasso = cross_val_score(best_lasso.best_estimator_, x_train, y_train, cv = 10)
print("LASSO // The cross-validated R2-score is : ", scores_lasso.mean())
print("LASSO // The standard deviation is : ", scores_lasso.std())

RIDGE // The cross-validated R2-score is :  0.9372466116438914
RIDGE // The standard deviation is :  0.03403792410422212
---
LASSO // The cross-validated R2-score is :  0.936968147667859
LASSO // The standard deviation is :  0.03454820221698877


Concernant Ridge, les performances ont été significativement améliorées par rapport au modèle Ridge classique sans ajustement des hyperparamètres.

Concernant Lasso, on observe peu d'amélioration des performances en ajustant les hyperparamètres.

En appliquant les hyperparamètres, les résultats sont très proches entre Ridge et Lasso, aussi bien sur les ensembles d’entraînement que de test.

La validation croisée confirme cette observation avec des scores moyens très similaires.

---

Sauvegarde des résultats dans un dataframe

In [18]:
scores_df = pd.DataFrame(columns = ["model", "R2"])
new_rows = [{"model": "best_ridge", "R2": best_ridge.score(x_test, y_test)},
            {"model": "best_lasso", "R2": best_lasso.score(x_test, y_test)}]
scores_df = pd.concat([scores_df, pd.DataFrame(new_rows)], ignore_index=True)
scores_df.to_csv("../datas/Walmart_Scores.csv", mode="a", header=False, index=False)

/tmp/ipykernel_1901/4017976665.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  scores_df = pd.concat([scores_df, pd.DataFrame(new_rows)], ignore_index=True)
